<font size="10" color="black">Análise Multivariada</font>

Eduardo Chaves Ferreira

## O que será tratado no curso


Análise multivariada



## Importação de bibliotecas usadas nos exemplos

In [ ]:
'''
As bibliotecas usadas são:
random
statistic
numpy.random
scipy.stats
pandas
matplotlib
statsmodels
pandas-profiling
'''

import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
import scipy.stats as stat

import os

path = os.environ['PATH']

if path.startswith('C'):
    IN_KAGGLE = False
else:
    IN_KAGGLE = True

## Mantendo a reprodutibilidade dos resultados

Antes da geração de números aleatórios é importante inicializar o gerador de números para que os resultados sejam os mesmos

In [ ]:
# Para uso com funções da biblioteca standard (ex random.randint)
random.seed(1)
# Para uso com funções da biblioteca numpy (ex np.random.randint)
np.random.seed(1)

# Quando for passada como parâmetro a seed
random_state = 1

## Funções gerais usadas nos exemplos

In [ ]:
# Permutação: possibilidades de colocação de n objetos em n posições = n!
def permutacao (n):
    return math.factorial(n)

# Arranjo: p objetos em n posições, ordem importa = n!/(n-p)!
def arranjo (n,p):
    return math.factorial(n)/math.factorial(n-p)

# Combinação: p objetos em n posições, ordem não importa = n!/(n-p)!p!
def combinacao (n,p):
    return math.factorial(n)/(math.factorial(n-p)*math.factorial(p))

# Variações possíveis havendo n slots e p possibilidades para cada um
def possibilidades(n,p):
    return p**n

<font size="6" color="red">Correlação entre Variáveis</font>

A correlação analisa possíveis relacionamentos existentes entre variáveis distintas. Procura determinar se são independentes ou dependentes e, no último caso, qual o tipo de dependência.

Vamos a um exemplo: o relatório de felicidade por pais da ONU, que mede o índice de felicidade por nação e seus indicadores (fonte: https://www.kaggle.com/unsdsn/world-happiness).


In [ ]:
if IN_KAGGLE:
    df = pd.read_csv("../input/2017.csv")
else:
    df = pd.read_csv("2017.csv")
    

df.head(20)

In [ ]:
df = df.loc[:,[  'Happiness.Score',  'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.','Dystopia.Residual']]

In [ ]:
plt.figure(figsize=(14,6))

_ = df['Happiness.Score'].hist( bins=50, density=True)

plt.xlabel('Índice de felicidade')
plt.ylabel('Probabilidade')
plt.title('Histogram do índice de felicidade')

plt.show()

Vamos calcular o índice de correlação entre as variáveis do relatório.

Uma forma visual e simples de analisar correlações é através de scatter plots.

In [ ]:
df.corr()

In [ ]:
# Aparentemente economia é fortemente correlacionada com felicidade

_ = df.plot(figsize=(14,6), kind='scatter', x='Happiness.Score', y='Economy..GDP.per.Capita.' )

In [ ]:
# Já a generosidade não apresenta correlação significativa

plt.figure()

_ = df.plot(figsize=(14,6),kind='scatter', x='Happiness.Score', y='Generosity' )

In [ ]:
from pandas.plotting import scatter_matrix

_ = scatter_matrix(df, figsize=(14,10), alpha=0.2, diagonal='kde')


Mais formalmente, as medidas mais usadas de correlação são:

Covariância: mede a variação conjunta das duas variáveis em torno de suas médias

Coeficiente de correlação de Pearson: mede a variação conjunta das variáveis em torno da média, variações normalizadas pelo desvio padrão (mais fácil interpretação porque o resultado fica no intervalo [-1 1])

Coeficiente de correlação de Spearman: calcula o coeficiente de correlação de Pearson utilizando rank order dos elementos dos arrays

In [ ]:
#df.plot(figsize=(14,6), kind='scatter', x='Happiness.Score', y='Economy..GDP.per.Capita.' )
np.cov(df[['Happiness.Score','Economy..GDP.per.Capita.']].values.T)

In [ ]:
#df.plot(figsize=(14,6), kind='scatter', x='Happiness.Score', y='Economy..GDP.per.Capita.' )
np.corrcoef(df[['Happiness.Score','Economy..GDP.per.Capita.']].values.T)

In [ ]:
from scipy.stats import spearmanr

rho, pval = spearmanr(df[['Happiness.Score','Economy..GDP.per.Capita.']].values)
print(rho)
print(pval)

## Correlação não linear

As correlações analisadas até agora são lineares, indicando se ambas as variáveis tem igual variação em torno da média.

Em correlações não lineares o pressuposto acima não se configura, embora haja correlação entre as variáveis.

Por isso há que se ter cuidado com a análise dos indicadores acima.

In [ ]:
x=np.arange(0, 8.1, 0.05)
y = np.sin(np.pi*x)

fig, ax1 = plt.subplots(figsize=(14,8))
ax1.plot( range(len(x)),y)
plt.grid()
plt.tight_layout()
plt.show()

print(len(x))

print(np.corrcoef(x,y))

## Autocorrelação

É a correlação feita com a própria variável, considerando um intervalo, normalmente tempo.

Assim, a série é sobreposta a ela mesma, inicialmente com deslocamento 0, depois 1, depois 2, ...

Os deslocamentos seguem até que não haja mais sobreposição dentre as séries.

O valor é decrescente porque, a medida em que ocorre o deslocamento, a sobreposição diminui.

É um excelente teste de sazonalidade em séries temporais.

In [ ]:
# https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/

from statsmodels.graphics.tsaplots import plot_acf

fig, ax = plt.subplots(1,1,figsize=(14,8))
plot_acf(y,ax=ax)

plt.show()

## Correlação não significa causalidade

A existência de correlação entre variáveis não significa, necessariamente, que uma variável afete diretamente o valor da outra.

A correlação indica que há movimentação conjunta de ambas as variáveis, efeito que pode ser produzido por uma terceira variável.

No exemplo abaixo, ambas as variáveis (sen e cos) estão fortemente correlacionadas (-1). Porém, o que afeta o valor da ambas é o ângulo original.

Para exemplos concretos, ver o link abaixo.

http://www.tylervigen.com/spurious-correlations


In [ ]:
x=np.arange(0, 8.1, 0.05)
y1 = np.sin(np.pi*x)
y2 = np.cos(np.pi*(x+1/2))

fig, ax = plt.subplots(2,1,figsize=(14,8))
ax[0].plot(y1)
ax[1].plot(y2)
plt.tight_layout()
plt.show()


print(np.corrcoef(y1,y2))

<font size="6" color="red">Anexo I - Funções Úteis</font>

## Gerando números aleatórios


In [ ]:
# Gerando int - biblioteca python standard
print(random.randrange(100, 1000, 2))
print(random.randint(100, 1000))

# Gerando int - biblioteca numpy
print(np.random.randint(100, 1000,2))

# Gerando float - biblioteca python standard
print(random.random())
print(random.uniform(100, 1000))
print(random.normalvariate(1, 1))

# Gerando float - biblioteca numpy
print(np.random.random(5))
print(np.random.randn(5))

np.random.random_sample(size=100)

## Gerando números não aleatórios

In [ ]:
print(np.linspace(0.0,1.0,11))
print(np.arange(0.0,10.0,3))
print(np.logspace(0.0,10.0,3))


## Escolha

In [ ]:
# Escolha com reposição
# usando numpy np.random.choice(10,size=10,replace=True)


faces = list(range(1,7))
lancamentos = 600
pesos = [1/6,1/6,0.5/6,0.5/6,2/6,1/6]
resultados = random.choices(population=faces, weights=pesos, k=lancamentos)
#print(resultados)
for i in faces:
    print('Face {}, peso {}, vezes {}'.format(i,pesos[i-1],resultados.count(i)))

In [ ]:
# Escolha sem reposição
# usando numpy np.random.choice(10,size=10,replace=False)


lista = list(range(1,7))
random.sample(population=lista, k=len(lista))


## Embaralhamento

In [ ]:
# Embaralhamento
# usando numpy np.random.choices

lista = list(range(1,7))
random.shuffle(lista)
lista

<font size="6" color="red">Anexo II - Referências</font>

Tutoriais

https://www.youtube.com/watch?v=Iq9DzN6mvYA

https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/

http://nbviewer.jupyter.org/url/norvig.com/ipython/Probability.ipynb

https://www.youtube.com/watch?v=KhAUfqhLakw

https://www.analyticsvidhya.com/blog/2017/09/6-probability-distributions-data-science/

https://www.datacamp.com/community/tutorials/python-statistics-data-science

https://machinelearningmastery.com/


Distribuições de probabilidade

http://blog.cloudera.com/blog/2015/12/common-probability-distributions-the-data-scientists-crib-sheet/

http://www.math.wm.edu/~leemis/chart/UDR/UDR.html

Cursos

https://courses.edx.org/courses/course-v1:UCSanDiegoX+DSE210x+1T2018/course/#block-v1:UCSanDiegoX+DSE210x+1T2018+type@chapter+block@c1c0e5a497924a40b800bf69e96b4004

Documentação bibliotecas Python

https://docs.python.org/3/library/statistics.html

https://docs.python.org/3/library/random.html

Documentação bibliotecas SciPy

https://docs.scipy.org/doc/scipy/reference/stats.html

Documentação bibliotecas NumPy

https://docs.scipy.org/doc/numpy/reference/routines.random.html

https://docs.scipy.org/doc/numpy/reference/routines.statistics.html

Dataframe

http://pandas.pydata.org/pandas-docs/version/0.13/visualization.html
